# Data Science for Public Good: MCMC Hands On Example



## Begin importing and installing necessary Python packages

In [ ]:
#Import important modules that we'll need to run this example
  #numpy: One of the best and most useful science packages available in Python
import numpy as np

  #matplotlib: The most common graphing Python package
import matplotlib.pyplot as plt
from matplotlib import colors, colorbar

  #Still matplotlib, but sub-packages needed for animations
from matplotlib import rc
rc('animation', html='jshtml')
import matplotlib.animation as animation


In [ ]:
#These are some backend commands that you don't need to worry about
  #This changes the font size on our plots
plt.rcParams['font.size'] = 14

  #This lets Google Colab make long videos
plt.rcParams['animation.embed_limit'] = 2**128

In [ ]:
#Not all Python packages come loaded on Google Colab (or your local computer)! You will
#most likely need to install them. The easiest way is by using pip (if you have a normal
#Python installation) or 'conda install' (if you use anaconda).

  #Google Colab uses pip as its package manager, so install the emcee and corner packages
!pip install emcee corner

  #NOTE: If you run this on your local machine, the above command will not work. You can use
  #pip in a command line or conda install (depending on your Python installation) to install these
  #packages
  
  #emcee: Python-based MCMC installation by Dan Foreman-Mackey
import emcee

  #corner: Makes corner plots of our parameters...more later!
import corner

## A more in-depth example of MCMC for a linear data set we make (Not covered during the hands-on example)

Let's begin by making a linear data set that we will use for this example:

In [ ]:
#Choose values for the slope (m) and intercept (b) below
m, b = 1, 2

#Choose an (integer) number of data points to generate our data set
N = 50

#Randomly choose N numbers between 0 and 10 (exclusive)
x_values = np.sort(10 * np.random.rand(N))

#Use m and b above to construct our data set
y_values = m * x_values + b

Now, let's plot the data we just made:

In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(x_values, y_values, 'ro')
plt.xlabel("x")
plt.ylabel("y")
plt.title("Test Data")

Noise (or statistical error) is an issue that affects all quantitative data. All measurements will have some error that cannot (and will not) ever go away. For this example, let's add random (or Gaussian) noise to our data

In [ ]:
#Choose N numbers from 0 to 1
delta_y = np.random.rand(N)

#Calculate the shift (or noise) in our "measurement" by adding Gaussian noise
shift = delta_y * np.random.normal(0, 1, N)

#Now actually shift the data
y_values += shift

Let's plot the data again, but, this time, the data have "noise" added to them:

In [ ]:
plt.figure(figsize=(6, 6))
plt.errorbar(x_values, y_values, delta_y, fmt='o', color='r')
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.title("Noisy Data")

Now, we have an issue. Originally, our data were nice and linear, and we could easily read off the slope and intercept. Now that we've added noise, it's not so easy to tell what values best fit the data. This is where model fitting comes in (e.g., linear regression). Here, however, we will use an MCMC process to fit the data.

In the presentation, I gave a short intro into what MCMC. In the Mauna Loa example below, I go into much more detail about the (negative) log likelihood. We'll take the (negative) log likelihood function as described in the following example.

In [ ]:
#The preferred model parameters occur where the negative (* 1/2) chi^2 is minimized
def log_prob(params, x, y, yerr):
  m, b = params
  model = m * x + b
  return -0.5 * np.sum((y - model)**2.0 / yerr**2.0)

Now, we need to set up emcee to run our MCMC fitting. To do this, we need to define the number of walkers and their initial starting positions.

In [ ]:
#How many walkers do we want?
nwalkers = 100

#Where do you want to center the walkers in parameter space?
m_start, b_start = -9, 10

#Now determine the initial positions
positions = np.array([m_start, b_start]) + np.random.randn(nwalkers, 2)

The last step is determining how many steps you want to take. I've tested this on Google Colab. For the animations (below) to work, this should be set to $<500$. Luckily this is an easy example, so that many steps (may) be overkill.

In [ ]:
#Number of steps
steps = 500

#Make the sampler object
sampler = emcee.EnsembleSampler(nwalkers, 2, log_prob, args=(x_values, y_values, delta_y));

#And run it
sampler.run_mcmc(positions, steps, progress=True);

Now, we need to get the MCMC "chain." This is a fancy way of getting the location of each walker in time. Anecdotally, this is the same as a walking(or running or biking) tracker that records your location on Earth in time.

In [ ]:
chain = sampler.get_chain()

Now, let's make the trace plots for one walker in time.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 4))

#Plot the first walker's slope position in time
ax[0].plot(chain[:, 0, 0], 'b.-')
ax[0].axhline(m, ls='--', color='k')

#Plot the first walker's intercept position in time
ax[1].plot(chain[:, 0, 1], 'b.-')
ax[1].axhline(b, ls='--', color='k')

#Plot the first walker's position in parameter space
color = ax[2].scatter(chain[:, 0, 0], chain[:, 0, 1], c=np.arange(chain.shape[0]), cmap='jet')
ax[2].axvline(m, ls='--', color='k')
ax[2].axhline(b, ls='--', color='k')

#The color bar shows the step number (bluer is early, red is late)
cbar = plt.colorbar(color)
cbar.set_label("Step Number")

#These set the x and y labels
ax[0].set_xlabel("Step Number")
ax[1].set_xlabel("Step Number")

ax[0].set_ylabel("m")
ax[1].set_ylabel("b")

ax[2].set_xlabel("m")
ax[2].set_ylabel("b")

plt.tight_layout()

All of this code below is to make the animation of the first walker's position in time. Feel free to look at this. Note that if you used more than 1000 steps, this won't work (it will silently fail). This may take a few minutes, so sit back and relax...

In [ ]:
fig_anim, ax_anim = plt.subplots(1, 4, figsize=(20, 4), gridspec_kw={'width_ratios': [15, 15, 15, 1]});

m_line, = ax_anim[0].plot([], [], 'b.-', animated=True)
b_line, = ax_anim[1].plot([], [], 'b.-', animated=True)
mb_scatter = ax_anim[2].scatter([], [], animated=True)

cmap = plt.get_cmap('jet')

for i in range(2):
  ax_anim[i].set_xlim(0, chain.shape[0] + 1)

ax_anim[0].set_ylim(ax[0].get_ylim())
ax_anim[1].set_ylim(ax[1].get_ylim())

ax_anim[2].set_xlim(ax[2].get_xlim())
ax_anim[2].set_ylim(ax[2].get_ylim())

ax_anim[0].axhline(m, color='k', ls='--')
ax_anim[1].axhline(b, color='k', ls='--')
ax_anim[2].axvline(m, ls='--', color='k')
ax_anim[2].axhline(b, ls='--', color='k')

ax_anim[0].set_xlabel("Step Number")
ax_anim[1].set_xlabel("Step Number")

ax_anim[0].set_ylabel("m")
ax_anim[1].set_ylabel("b")

ax_anim[2].set_xlabel("m")
ax_anim[2].set_ylabel("b")

fig_anim.text(0.93, 0.5, "Step Number", rotation=90, va='center')
plt.tight_layout()

anim_lines = [m_line, b_line, mb_scatter]

cbar =  colorbar.ColorbarBase(ax_anim[3], cmap=cmap, norm=colors.Normalize(0, chain.shape[0]), orientation='vertical')
def frame(N):
  anim_lines[0].set_data(np.arange(N), chain[:N, 0, 0])
  anim_lines[1].set_data(np.arange(N), chain[:N, 0, 1])

  time_steps = cmap(np.linspace(0, 1, chain.shape[0]))

  anim_lines[2].set_offsets(np.c_[chain[:N, 0, 0], chain[:N, 0, 1]])
  anim_lines[2].set_facecolors(time_steps)

anim = animation.FuncAnimation(fig_anim, frame, frames=chain.shape[0], blit=False, repeat=True)
anim

Pretty cool, huh? While this makes a nice visual representation of what MCMC is doing, in reality, we always use more than 1 walker. In this case, we should make a trace plot using all of the walkers on one plot: 

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6), sharex=True)

#Plot all of the walkers' slope positions
ax[0].plot(chain[:, :, 0], 'k-', alpha=0.5);
ax[0].axhline(m, color='r', ls='--')

#Plot all of the walkers' intercept positions
ax[1].plot(chain[:, :, 1], 'k-', alpha=0.5);
ax[1].axhline(b, color='r', ls='--')

#Add labels
ax[0].set_xlabel("Step Number")
ax[1].set_xlabel("Step Number")

ax[0].set_ylabel("m")
ax[1].set_ylabel("b");


What we didn't see in the one walker trace plots is this "fuzzy" or widened trace that shrinks at high step numbers. This is a key indication that our fit has truly "converged." 

The "fuzzy" portion of this plot is where the walkers are generally traversing the parameter space but haven't found a valley yet. We only care about the parameter space where the walkers generally have started to find this valley. The time, or number of steps, it takes for this to occur is known as the "burn length." If we make it too high (relative to the number of steps), we won't have enough data points to sample the parameter space. If we make it too low, we're sampling parts of the parameter space that don't describe our data.

For this example, I'll choose a burn length of 200 steps. However, I highly suggest changing this number (below) and seeing how it effects the corner plots.

In [ ]:
#Choose a burn length
burn_length = 200

#Get the burned chain
burned_chain = sampler.get_chain(discard=burn_length, thin=1, flat=True)

Now, we get to make the all important corner plot that shows the histograms of our fitted parameters (known as posterior distributions) and the parameter space of the model.

In [ ]:
fig = corner.corner(burned_chain, labels=["m", "b"], truths=[m, b], quantiles=[0.16, 0.5, 0.84], show_titles=True, title_fmt=".3f")

fig.text(0.70, 0.75, "Posterior Distributions", fontsize=16)
fig.text(0.55, 0.725, "←", fontsize=60)
fig.text(0.65, 0.62, " ↓", fontsize=60)

Not bad...notice that the "truths" (blue lines) don't exactly match the parameters you set above. This is due to the statistical noise that we added. Try going back to the top of this example and changing the "size" of this noise to see how it affects the results.

The lower left panel shows the actual parameter space of the fit as sampled by the walkers.

### One last thing about MCMC (at least here, anyway...)

In a least squares regression, we can use "constraints" to tell the fitting algorithm where we know (AKA a priori) the parameters cannot be. For example, even in the noisy data set, we know if the slope is either positive or negative. We can specifically tell (most) least squares algorithms where not to look.

We can do the same thing in MCMC, but these constraints (known as priors) are more powerful. Since everything in MCMC fitting is a distribution, we can tell the program exactly where to (and where not to) look. For example, let's say we know that the slope is positive. In emcee, we can write the prior function as:

In [ ]:
#Our new prior
def log_prior_new(params):
  m, b = params
  if m >= 0:
    return 0.0
  else:
    return -np.inf

#Our new likelihood function with priors taken into account
def log_prob_new(params, x, y, yerr, prior):
  m, b = params
  model = m * x + b
  return -0.5 * np.sum((y - model)**2.0 / yerr**2.0) + prior(params)

Notice that we return $0$ or $-\infty$ from our prior function. $0$ is returned when the slope is $>0$ (which we want), and this does not affect our (negative) log likelihood. When the slope is outside of this constraint, it returns $-\infty$ and tells the MCMC program to pick a different set of parameters.

How does this affect our results?

In [ ]:
#Where do you want to center the walkers in parameter space?
  #Note that the starting values you choose MUST be where the prior is valid otherwise it won't work properly
m_start, b_start = 9, 10

#Now determine the initial positions
positions = np.array([m_start, b_start]) + np.random.randn(nwalkers, 2)

#Run MCMC again
sampler_prior = emcee.EnsembleSampler(100, 2, log_prob_new, args=(x_values, y_values, delta_y, log_prior_new));
sampler_prior.run_mcmc(positions, steps, progress=True);

chain = sampler_prior.get_chain()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 4))
ax[0].plot(chain[:, 0, 0], 'b.-')
ax[0].axhline(m, ls='--', color='k')

ax[1].plot(chain[:, 0, 1], 'b.-')
ax[1].axhline(b, ls='--', color='k')

color = ax[2].scatter(chain[:, 0, 0], chain[:, 0, 1], c=np.arange(chain.shape[0]), cmap='jet')
ax[2].axvline(m, ls='--', color='k')
ax[2].axhline(b, ls='--', color='k')

cbar = plt.colorbar(color)
cbar.set_label("Step Number")

ax[0].set_xlabel("Step Number")
ax[1].set_xlabel("Step Number")

ax[0].set_ylabel("m")
ax[1].set_ylabel("b")

ax[2].set_xlabel("m")
ax[2].set_ylabel("b")

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6), sharex=True)
ax[0].plot(chain[:, :, 0], 'k-', alpha=0.5);
ax[0].axhline(m, color='r', ls='--')

ax[1].plot(chain[:, :, 1], 'k-', alpha=0.5);
ax[1].axhline(b, color='r', ls='--')

ax[0].set_xlabel("Step Number")
ax[1].set_xlabel("Step Number")

ax[0].set_ylabel("m")
ax[1].set_ylabel("b");

In [ ]:
burned_chain = sampler.get_chain(discard=burn_length, thin=1, flat=True)

fig = corner.corner(burned_chain, labels=["m", "b"], truths=[m, b], quantiles=[0.16, 0.5, 0.84], show_titles=True, title_fmt=".3f")

Did anything really change in this example? No, but that's because a linear regression is very easily fit using MCMC. For more complicated models/data, the use of priors can be extremely important.

**Question: Why do we need priors at all?**

For this toy model, we know certain properties of the data already (i.e., the slope and y-intercept must both be greater than 0). Since negative values for either parameter won't fit the data well, we don't have to probe those ranges at all. 

In everyday science use, however, priors can be used in several ways:


1.   Removing unphysical values altogether (i.e., $T < 0~\text{Kelvin}$)
2.   Using previously determined parameters from different studies (i.e., someone previously calculating an exoplanet's mass of $1.2\pm0.1$ Jupiter masses)
3.  Best guesses about what the parameter values might be (i.e., "the slope looks like it's between 2 and 20")

In cases 2 & 3, adding more data to your fits will reduce the effect that your priors have on your posterior distribution. 

*Overall, we use prior distributions to reflect what we already know about the data/system.*


## Mauna Loa CO$_2$ Data Set: Hands-On

In [ ]:
#Let's load in the CO2 data set...
time, co2_ppm, co2_err = np.loadtxt("co2_weekly_mlo.csv", usecols=(3, 4, 5), unpack=True, skiprows=1, delimiter=',')

In [ ]:
#...and plot it!
plt.figure(figsize=(6, 6))
plt.plot(time, co2_ppm)
plt.xlabel("Year")
plt.ylabel("CO$_2$ concentration [ppm]")
plt.title("Mauna Loa Monitoring Station: Uncleaned")

Uh oh...there are some points where the data are missing and have been filled with -1000. Let's remove those first:

(see more about data cleaning in Alex's activity)

In [ ]:
#Find where the concentration data is < 0 (where they are unphysical)...
unphys_mask = np.less(co2_ppm, 0)

#...and delete them from the time and concentration arrays.
time = np.delete(time, unphys_mask)
co2_ppm = np.delete(co2_ppm, unphys_mask)
co2_err = np.delete(co2_err, unphys_mask)

In [ ]:
#Now, plot the fixed data:
plt.figure(figsize=(6, 6))
plt.plot(time, co2_ppm)
plt.xlabel("Year")
plt.ylabel("CO$_2$ concentration [ppm]")
plt.title("Mauna Loa Monitoring Station: Cleaned")

Much better. The next thing we need to decide on is what kind of function we'll use to fit the data. There are two distinct features of the data:


1.   An oscillating (or periodic) modulation
2.   A base function that is increasing faster than a linear function

So, I suggest we use:
$$\text{CO}_2(t) = at^2+bt+\text{amp}\cdot\cos(2\pi t+\phi)+c$$ 

#### Question: How many parameters do we have?

##### Answer

 5 (a, b, c, amp, $\phi$)

#### Log Likelihood function

In linear least squares, we minimize the summed, squared residual. In MCMC, we minimize a function called the (negative) log likelihood:

$$\mathcal{L}=-\dfrac{1}{2}\sum\left(\dfrac{(\text{data} - \text{model})^2}{\text{error}^2}\right)$$

This is similar to the summed, squared residual, but there are two changes:


1.   There's a $-\frac{1}{2}$ in front of the sum
2.   We divide the squared residual by the error/uncertainty of our data

(This is commonly known as  $-\frac{1}{2}\chi^2$)





#### Question: Why do we need to divide by the error?

##### Answer: $\mathcal{L}$ needs to be unitless, while the summed, squared residual can be have a unit. 

 #### Let's write the log likelihood function

In [ ]:
#Calculate the (negative) log likelihood
def log_prob(params, t, y, yerr, prior_func):

  #This line shifts the time relative to January 1, 1970
  #This is NOT necessary at all. It makes the calculation
  #finish/converge quicker
  t0 = t - 1970.

  #These 5 parameters are chosen using a random walk
  a, b, c, amp, phase= params

  #Calculate the model function using these 5 parameters
  model = a * t0**2.0 + b * t0 + c + amp * np.cos(2.0 * np.pi * t0 + phase)
  
  #Calculate L and return the value
  return -0.5 * np.sum(((y - model)/ yerr)**2.0) + prior_func(params)

  #This is a prior function (which I don't have time to go over)! 
  #See the more in-depth example above
def log_prior(params):
  a, b, c, amp, phase = params

  #Only allow each data point to have a phase between 0 and 2 pi
  #The phase is degenerate above this
  if phase >= 0 and phase <= 2 * np.pi:
    return 0.0
  return -np.inf

#### Now, we have to the parameters we need to define in order to use emcee

In [ ]:
#ndim is the number of parameters (dimensions) we're fitting
ndim = 5

#### Question: Is there such a thing as "too many" or "too few" walkers?

##### Answer: Yes. If you add too many walkers, the program will run very slowly. If you don't add enough, you will need to run more steps to get a good sample. There is no defined or accepted way to determine how many to use.

### Define the number of walkers and determine their random starting points

In [ ]:
  #nwalkers is the number of MCMC walkers we want
nwalkers = 500

  #Randomly sample the parameter space around a = 0, b = 1, c = 300, amp = 5, and phi = 0
  #NB: I fine tuned these values so that the MCMC algorithm would converge very quickly
  #Suggestion: Change these parameters (after we send this notebook out) to see how
  #the initial sampling point changes the number of steps required for convergence
positions = np.array([0, 1, 300, 5, 0]) + np.random.randn(nwalkers, ndim)

### Feed emcee all the parameters/functions we've defined and run it

In [ ]:
#How many steps should we run?
steps = 2000

#Create the sampling object...
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob, args=(time, co2_ppm, co2_err, log_prior));

#...and run MCMC!
sampler.run_mcmc(positions, steps, progress=True);

### Trace plots

In [ ]:
#In MCMC, a "chain" is a record of each walker's steps in time.
#Visualizing the chain is a good way to determine if the MCMC
#has "converged"
chain = sampler.get_chain()

In [ ]:
#Plotting the traces for all walkers using all steps
fig, ax = plt.subplots(ndim, 1, figsize=(12, 12), sharex=True)
labels = ["a [ppm/yr$^2$]", "b [ppm/yr]", "c [ppm]", "amp. [ppm]", "$\\phi$ [rad]"]

for i in range(ndim):
  ax[i].plot(chain[:, :, i], 'k-', alpha=0.5);
  ax[i].set_ylabel(labels[i])

ax[-1].set_xlabel("Step Number")
plt.subplots_adjust(hspace=0)

Trace plots are a good way to determine if an MCMC run has converged. In the first few steps, the plots are "fuzzy", but they eventually become compact. The rough time where this occurs is known as the "burn length."

Question: Where (approximately) should we choose the burn length for these plots?

In [ ]:
burn_length = 300

### More advanced topic: How do we determine if a chain has converged?

#### The most common way is by using the "autocorrelation" time. In not-so-technical terms, this is number of steps required for a walker to "forget" where it was. emcee has a nice set of tutorials (with math) to explain this. 

https://emcee.readthedocs.io/en/stable/tutorials/autocorr/

### Corner Plots

One of (if not the most) useful MCMC plots is known as the corner plot. On the diagonal are the histograms (known as posterior distributions) of the 5 fitted parameters. The other panels show the locations of the walkers for each pair of parameters. This is similar to the "parameter space" plots we saw in the presentation, but in more dimensions.

In [ ]:
burned_chain = sampler.get_chain(discard=burn_length, thin=1, flat=True)

fig = corner.corner(burned_chain, quantiles=[0.16, 0.5, 0.84], show_titles=True, title_fmt=".3f", titles=labels, title_kwargs={'fontsize' : 11.5})

This data set shows the amplitude and $\phi$ have a double valley parameter space! We would not have known/seen this if we used least squares regression.

### Question: Can you think of the reason why there are two valleys in the amplitude-$\phi$ parameter space?

#### Answer: The phase shift, $\phi$, has two peaks: $\approx1.57, 4.71$. These are close to the value required to change $\cos x$ into $\sin x$ terms (i.e., $\cos(x+1.57) \approx -\sin x$ and $\cos(x+4.71)\approx\sin x$). Simply, this tells us that we should have used $\sin$ instead of $\cos$ in our function.